In [2]:
import numpy
import pandas as pd

In [3]:
df = pd.read_csv('raw_data.csv', encoding='utf-8-sig')

In [13]:
# 행 번호 삭제
df.drop(df.columns[0], axis=1)

'1000대-현대자동차그룹-안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-매출 1조 클럽-HOT한 인기기업-평균연봉 6천이상----'

In [19]:
tmp = str(df['pros'][5])
list = tmp.split('-')
list
count = 0
for elem in list:
    if elem != '':
        count+=1
print(list)
print("-> " + str(count))

['1000대', '현대자동차그룹', '안정적인 장수 회사', '성장판 열린 회사', '구직자 인기기업', '매출 1조 클럽', 'HOT한 인기기업', '평균연봉 6천이상', '', '', '', '']
-> 8
